In [1]:
import torch.optim as optim
import torch.nn as nn
from qiskit.providers.fake_provider import FakeLima
from scripts.data_setup import load_data
from scripts.from_circ_to_numpy import operations_to_features, save_to_json, load_from_json
from scripts.model_ann_first import create_models, train_and_test_step, save_models, load_models

In [2]:
n_qubits = 5 # 5 == n_qubits, default value for now because of FakeLima
train_circuits, train_observables, train_ideal_exp_vals, train_noisy_exp_vals, test_circuits, test_observables, test_ideal_exp_vals, test_noisy_exp_vals = load_data('data/circuits/data_small_1')

In [3]:
backend = FakeLima()
n_qubits = 5

In [4]:
X_train, y_train, X_test, y_test = load_from_json('data/features/data_small_1')

In [5]:
# specify these :)
sequence_input_size = 7
sequence_hidden_size = 2
sequence_num_layers = 1
sequence_model_type = 'LSTM'
sequence_dropout = 0
ann_hidden_layers = 2
ann_hidden_units = 64
ann_dropout = 0.1
first_ann_hidden_layers = 2
first_ann_dropout = 0
noisy_first = True

sequence_model, ann, first_ann = create_models(sequence_input_size, 
                                                sequence_hidden_size, 
                                                sequence_num_layers, 
                                                sequence_model_type, 
                                                sequence_dropout, 
                                                ann_hidden_layers, 
                                                ann_hidden_units, 
                                                ann_dropout, 
                                                first_ann_hidden_layers=first_ann_hidden_layers,
                                                first_ann_p_dropout=first_ann_dropout,
                                                noisy_first=noisy_first)

first_ann_config = {
    "hidden_layers": first_ann_hidden_layers,
    "dropout": first_ann_dropout,
}

sequence_config = { 
    "input_size": sequence_input_size,
    "hidden_size": sequence_hidden_size,
    "num_layers": sequence_num_layers,
    "model_type": sequence_model_type,
    "dropout": sequence_dropout
}

ann_config = {
    "hidden_layers": ann_hidden_layers,
    "hidden_units": ann_hidden_units,
    "dropout": ann_dropout,
    "noisy_first": noisy_first
}

In [6]:
loss_fn = nn.MSELoss() # leave for now
optimizer = optim.Adam(list(ann.parameters()) + list(sequence_model.parameters()) + list(first_ann.parameters()), lr=0.0001)

num_epochs = 5
train_and_test_step(sequence_model, ann, first_ann, loss_fn, optimizer, X_train, train_noisy_exp_vals, y_train, X_test, test_noisy_exp_vals, y_test, num_epochs, noisy_first=noisy_first)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5, train loss: 1.0676, test_loss: 0.5102
Epoch 2/5, train loss: 0.3708, test_loss: 0.2755
Epoch 3/5, train loss: 0.3330, test_loss: 0.2272
Epoch 4/5, train loss: 0.3006, test_loss: 0.2315
Epoch 5/5, train loss: 0.3054, test_loss: 0.2232


In [7]:
save_models(sequence_model=sequence_model,
            ann=ann,
            first_ann=first_ann,
            sequence_config=sequence_config, 
            ann_config=ann_config,
            first_ann_config=first_ann_config,
            save_dir='data/models/model_test_3')

Models and configurations saved successfully.


In [8]:
sequence_model, ann, first_ann = load_models('data/models/model_test_3')

Models and configurations loaded successfully.
